In [1]:
!pip install fastai==1.0.6
from fastai import *        # Quick access to most common functionality
from fastai.text import *   # Quick access to NLP functionality

    100% |████████████████████████████████| 102kB 5.1MB/s 
    100% |████████████████████████████████| 23.3MB 2.6MB/s 
    100% |████████████████████████████████| 614kB 12.6MB/s 
    100% |████████████████████████████████| 92kB 22.4MB/s 
    100% |████████████████████████████████| 1.9MB 14.8MB/s 
  Running setup.py bdist_wheel for regex ... done
  Stored in directory: /root/.cache/pip/wheels/74/17/3f/c77bba99efd74ba1a19862c9dd97f4b6d735e2826721dc00ff
Successfully built regex
  Found existing installation: cymem 1.31.2
    Uninstalling cymem-1.31.2:
      Successfully uninstalled cymem-1.31.2
  Found existing installation: regex 2017.4.5
    Uninstalling regex-2017.4.5:
      Successfully uninstalled regex-2017.4.5
  Found existing installation: preshed 1.0.1
    Uninstalling preshed-1.0.1:
      Successfully uninstalled preshed-1.0.1
  Found existing installation: thinc 6.10.3
    Uninstalling thinc-6.10.3:
      Successfully uninstalled thinc-6.10.3
  Found existing installation: spac

# Text example

An example of creating a language model and then transfering to a classifier.

In [4]:
imdb_mount = '/floyd/input/imdb_reviews/'
wt103_mount = '/floyd/input/wt103_v1/'

In [16]:
!rm -rf data/
def build_directory_from_mounts(src='/floyd/input/imdb_reviews_wt103', dst='data/', sample_size=1000):
    """
    Copy mounted directory into writable local, then overwrite training language model csv with the specified sample_size
    """
    shutil.copytree(src, dst)
    dftr_lm = read_csv_with_sample_size(
        dst+'train.csv', sample_size=sample_size)
    dftr_lm.to_csv(dst+'train_lm.csv', index=False)
build_directory_from_mounts()

In [15]:
def read_csv_with_sample_size(path, sample_size=10, chunksize=24000):
  total_rows = get_total_length(path, chunksize=chunksize)
  frac = sample_size/total_rows
  df = pd.DataFrame()
  for chunk in pd.read_csv(path, chunksize=chunksize, header=None):
    df = pd.concat([df,chunk.sample(frac=frac)])
  return df

In [7]:
dftr = read_csv_with_sample_size(imdb_mount+'train.csv', sample_size=1000)
dftr_clas = pd.read_csv(dir_path+'train_clas.csv', header=None)
dftr.head()

,0,1
11849,0,Maggie is a widow in her fifties living in a s...
13857,0,I am appalled to see that so many people have ...
2765,0,"Barney is about ""IMAGINATION"" what you guys do..."
577,0,"Good characters, well paced, with a tight scri..."
12366,0,"This movie tackled closure, which is probably ..."


In [4]:
path = Path('data')
path

PosixPath('data/csv')

In [9]:
dftr.to_csv(path/'train.csv', index=False)
dftr_clas.to_csv(path/'train_clas.csv', index=False)

Open and view the independent and dependent variables:

In [10]:
df = pd.read_csv(path/'train.csv', header=None)
df.head()

,0,1
0,0,1
1,0,Maggie is a widow in her fifties living in a s...
2,0,I am appalled to see that so many people have ...
3,0,"Barney is about ""IMAGINATION"" what you guys do..."
4,0,"Good characters, well paced, with a tight scri..."


In [11]:
len(df)

1002

In [5]:
classes = read_classes(path/'classes.txt')[:2]
classes

['neg', 'pos']

Create a `DataBunch` for each of the language model and the classifier:

In [13]:
data_lm = TextLMDataBunch.from_csv(path, train="train_lm")
data_clas = TextClasDataBunch.from_csv(path, train='train_clas', valid='valid_clas', vocab=data_lm.train_ds.vocab)

Tokenizing train.


Numericalizing train.
Tokenizing valid.


Numericalizing valid.
Tokenizing train_clas.


Numericalizing train_clas.
Tokenizing valid_clas.


Numericalizing valid_clas.


[fast.ai](http://www.fast.ai/) has a pre-trained English model available that we can download.

In [14]:
!ls {path}

aclImdb      imdb_clas	models	train.csv	valid.csv
classes.txt  imdb_lm	tmp	train_clas.csv	valid_clas.csv


We'll fine-tune the language model:

In [15]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'])
learn.unfreeze()
learn.fit(2, slice(1e-4,1e-2))

Total time: 03:41
epoch  train loss  valid loss  accuracy
0      4.896679    4.185995    0.244655  (01:50)
1      4.635169    4.129789    0.249731  (01:50)



Save our language model's encoder:

In [16]:
learn.save_encoder(f'enc{len(df)}')

Fine tune it to create a classifier:

In [20]:
learn = RNNLearner.classifier(data_clas)
learn.load_encoder(f'enc{len(df)}')
learn.fit(3, 1e-3)

Total time: 10:48
epoch  train loss  valid loss  accuracy
0      0.651802    0.645724    0.612720  (03:36)
1      0.635723    0.566419    0.717680  (03:36)
2      0.632057    0.566731    0.705840  (03:35)



In [18]:
learn.save_encoder(f'enc_clas{len(df)}')